In [0]:
!pip install -U -q PyDrive
!pip install h5py pyyaml
!pip install -U pillow
!pip install tf_nightly
'''
import os
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
!pip uninstall torch
!pip uninstall torchvision
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
#!pip install torch==0.4.1 -f https://download.pytorch.org/whl/cu100/stable
!pip install torchvision[cuda10_package] torch[cuda10_package]
'''
#!pip install -q torch==1.1.0 torchvision
#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
#!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0.post4-cp36-cp36m-linux_x86_64.whl 
#!pip3 install torchvision
#!pip3 install torch torchvision
#!pip install -q torch==1.0.0 torchvision==0.4.1
'''
!pip install torch
!pip install torchvision

import torch
import torchvision
'''
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1BMYFlQ6yQRjjmi8P6vNU3Nt1sKOplM40'})
download.GetContentFile('Google.zip')
!unzip Google.zip

Requirement already up-to-date: pillow in /usr/local/lib/python3.6/dist-packages (6.1.0)
Archive:  Google.zip
replace Google/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Google/.DS_Store        
  inflating: __MACOSX/Google/._.DS_Store  
  inflating: Google/gray_train/79.Chinese_5a.jpg  
  inflating: Google/gray_train/147.gettyimages-1147112866.jpg  
  inflating: Google/gray_train/55.confused-man-640x396.jpg  
  inflating: Google/gray_train/112.500_F_224402640_xifjIRtaKuirDLoQCe8jGox4uaav63Wu.jpg  
  inflating: Google/gray_train/83.person-of-interest-e1517600346957.jpg  
  inflating: Google/gray_train/97.zuckerberg.jpg  
  inflating: Google/gray_train/326.the-more-i-think-the-more-confused-i-get-quote-2.jpg  
  inflating: Google/gray_train/100.khn-uribe-04-2_wide-ac7bde7fb295d279b18cddfe4f93259786c30953.jpg  
  inflating: Google/gray_train/363.signs-and-symptoms-of-dementia-800.jpg  
  inflating: Google/gray_train/57.353d9dacafcff0bd327636716792380f_confused-person-c

In [0]:
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
#from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
#from keras.utils import to_categorical
#from tensorflow.keras.preprocessing import image
from keras.preprocessing import image
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from tqdm import tqdm
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
train = pd.read_csv('Google/new_dataset.csv')
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
  try:
    img = image.load_img('Google/gray_train/'+train['id'][i], target_size=(154,154,1), color_mode="grayscale")
    #img = Image.open('Google/train/' + train['id'][i])
    #img = cv2.imread('Google/train/' + train['id'][i], 0)
    #new_img = cv2.resize(img, (154, 154))
    #img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #gray = img.convertTo(mode="L", dither=None, palette=0, colors=256)
    img = image.img_to_array(img)
    #img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = img/255
    train_image.append(img)
  except OSError:
    continue
X = np.array(train_image)

y=train['label'].values
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(154,154,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
# evaluate the model
scores = model.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model.save('newest-model.h5')
model_file = drive.CreateFile({'title' : 'newest-model.h5'})
model_file.SetContentFile('newest-model.h5')
model_file.Upload()
# download to google drive
drive.CreateFile({'id': model_file.get('id')})
#files.download('emotion_model.h5')
'''
download = drive.CreateFile({'id': '14gpcUHRRE0wRx4GvaXn2-rwXNR84wo6s'})
download.GetContentFile('Test.zip')
!unzip Test.zip
test = pd.read_csv('Test/frame_test.csv')
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('Test/Test/'+test['id'][i], target_size=(154,154,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)
# making predictions
prediction = model.predict_classes(test)
download = drive.CreateFile({'id': '19rCo83Xk9_nvZ7twlw6rvR6e_M3HpZEz'})
download.GetContentFile('sample_submission.csv')
# creating submission file
sample = pd.read_csv('sample_submission.csv')
sample['label'] = prediction
sample.to_csv('sample_cnn.csv', header=True, index=False)
'''

Using TensorFlow backend.
W0813 03:07:30.676886 140624450975616 __init__.py:689] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  
100%|██████████| 7533/7533 [00:04<00:00, 1675.38it/s]
W0813 03:07:36.669527 140624450975616 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1633: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0813 03:07:37.020078 140624450975616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/k

Train on 6026 samples, validate on 1507 samples
Epoch 1/10
6026/6026 [==============================] - 320s 53ms/sample - loss: 0.4699 - acc: 0.9003 - val_loss: 0.3256 - val_acc: 0.8978
Epoch 2/10
6026/6026 [==============================] - 317s 53ms/sample - loss: 0.3083 - acc: 0.9092 - val_loss: 0.3266 - val_acc: 0.8978
Epoch 3/10
6026/6026 [==============================] - 314s 52ms/sample - loss: 0.2982 - acc: 0.9094 - val_loss: 0.3379 - val_acc: 0.8978
Epoch 4/10
6026/6026 [==============================] - 312s 52ms/sample - loss: 0.2723 - acc: 0.9097 - val_loss: 0.3627 - val_acc: 0.8978
Epoch 5/10
6026/6026 [==============================] - 312s 52ms/sample - loss: 0.2433 - acc: 0.9137 - val_loss: 0.3481 - val_acc: 0.8958
Epoch 6/10
6026/6026 [==============================] - 310s 51ms/sample - loss: 0.1972 - acc: 0.9220 - val_loss: 0.4601 - val_acc: 0.8945
Epoch 7/10
6026/6026 [==============================] - 312s 52ms/sample - loss: 0.1698 - acc: 0.9346 - val_loss: 0.44

W0813 04:01:01.068187 140624450975616 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

"\ndownload = drive.CreateFile({'id': '14gpcUHRRE0wRx4GvaXn2-rwXNR84wo6s'})\ndownload.GetContentFile('Test.zip')\n!unzip Test.zip\ntest = pd.read_csv('Test/frame_test.csv')\ntest_image = []\nfor i in tqdm(range(test.shape[0])):\n    img = image.load_img('Test/Test/'+test['id'][i], target_size=(154,154,1), grayscale=True)\n    img = image.img_to_array(img)\n    img = img/255\n    test_image.append(img)\ntest = np.array(test_image)\n# making predictions\nprediction = model.predict_classes(test)\ndownload = drive.CreateFile({'id': '19rCo83Xk9_nvZ7twlw6rvR6e_M3HpZEz'})\ndownload.GetContentFile('sample_submission.csv')\n# creating submission file\nsample = pd.read_csv('sample_submission.csv')\nsample['label'] = prediction\nsample.to_csv('sample_cnn.csv', header=True, index=False)\n"